In [135]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import pyLDAvis.gensim
import pickle 
import pyLDAvis

ModuleNotFoundError: No module named 'pyLDAvis'

In [119]:
df = pd.read_csv('data/robloxParentsComments.csv')

In [120]:
df.head()

,url,author,date,timestamp,score,comment,comment_id,parent_id
0,https://www.reddit.com/r/RobloxParents/comment...,dianndianna,2021-05-06,1.620311e+09,3,I agree. I have been frustrated since download...,gx5jdbv,NaN
1,https://www.reddit.com/r/RobloxParents/comment...,cutenfuzzie,2021-05-06,1.620337e+09,3,I also agree. I am regretting ever letting my ...,gx79ppg,NaN
2,https://i.redd.it/wahei45uohz41.jpg,Deathly_banana,2020-06-11,1.591900e+09,3,Hehehe secy,ftq05wx,NaN
3,https://i.redd.it/wahei45uohz41.jpg,Afimiliarface,2020-07-01,1.593626e+09,2,I want to report this too what’s the game call...,fwlsxns,NaN
4,https://i.redd.it/wahei45uohz41.jpg,StupidSam1,2020-05-29,1.590754e+09,1,"Report it, these games are from around 2014 an...",fs6xz8e,NaN


In [121]:
df['comment']

0      I agree. I have been frustrated since download...
1      I also agree. I am regretting ever letting my ...
2                                            Hehehe secy
3      I want to report this too what’s the game call...
4      Report it, these games are from around 2014 an...
                             ...                        
166    No one really plays Murder Mystery 2 for the g...
167    Murder Mystery 2 is one of my favorite games o...
168                                Buy another computer.
169    Both games are in PC, Xbox, or Mobile. You can...
170    Have them take turns. Child 1 gets computer 2p...
Name: comment, Length: 171, dtype: object

In [122]:
df['comment'].str.count('game').sum()

110

In [123]:
text = " ".join(df['comment'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [124]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
print(bi_grams_list)

[('agree', 'frustrated'), ('frustrated', 'since'), ('since', 'downloading'), ('downloading', 'Roblox'), ('Roblox', 'son'), ('son', 'almost'), ('almost', '9'), ('9', 'months'), ('months', 'ago'), ('ago', 'would'), ('would', 'like'), ('like', 'able'), ('able', 'block'), ('block', 'games'), ('games', 'block'), ('block', 'content'), ('content', 'countries'), ('countries', 'due'), ('due', 'less'), ('less', 'monitoring'), ('monitoring', 'even'), ('even', 'make'), ('make', 'curated'), ('curated', 'list'), ('list', 'games'), ('games', 'think'), ('think', 'okay'), ('okay', 'games'), ('games', 'shown'), ('shown', 'allowed'), ('allowed', 'turn'), ('turn', 'account'), ('account', 'restrictions'), ('restrictions', 'basically'), ('basically', 'struggles'), ('struggles', 'find'), ('find', 'much'), ('much', 'anything'), ('anything', 'play'), ('play', 'gets'), ('gets', 'upset'), ('upset', 'going'), ('going', 'game'), ('game', 'game'), ('game', 'find'), ('find', 'let'), ('let', 'play'), ('play', 'really

In [125]:
pattern = r'^\s*(who|what|when|where|why|how)\b.*\?$'
questions = df['comment'].str.extract('({})'.format(pattern), flags=re.IGNORECASE)[0].dropna()

In [126]:
print(questions)

97    How long did it take to retain access to the r...
Name: 0, dtype: object


In [127]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: account restrictions with a frequency of 8


In [128]:
sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(feature_names):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                      term  rank
5    account restrictions   6.0
151     parental controls   3.0
79              game game   3.0
235       without account   3.0
142                nt get   2.0
49           curated list   2.0
158          play without   2.0


In [129]:
include_list = ['can','what','where','when','how','which','who','why']
df_ques = df['comment'].str.lower().str.contains('|'.join(include_list), na=False)
df_true = df.loc[df_ques]
print(df_true['comment'])

0      I agree. I have been frustrated since download...
3      I want to report this too what’s the game call...
4      Report it, these games are from around 2014 an...
11                   I dont know how he came to the game
12     This was Just a troll to see how this SR would...
                             ...                        
161    It helps keep my son occupied otherwise not su...
165    The only time it becomes a problem is if they ...
166    No one really plays Murder Mystery 2 for the g...
167    Murder Mystery 2 is one of my favorite games o...
169    Both games are in PC, Xbox, or Mobile. You can...
Name: comment, Length: 72, dtype: object


In [130]:
df_ques = df['comment'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['comment'])

4      Report it, these games are from around 2014 an...
9                                                      ?
13                                    clickbaited games?
83                 Can i offer Candy Set and logchopper?
94     I don’t really understand what happened, the v...
95     It was like a clip before the game opened (aft...
97     How long did it take to retain access to the r...
106    do the settings described [here](https://en.he...
139    Well your son has to learn.\nAl, he needs is a...
147              Hmm, have you check your local Walmart?
Name: comment, dtype: object


ModuleNotFoundError: No module named 'wordcloud'